Again making the heatmap, but this was just to save the vps, and then has a way to read them back in.

# Setup

In [4]:
# # NOT THIS ONE

# import pandas as pd  # Data handling
# from scipy.integrate import odeint # numerical integration
# import numpy as np
# from pyvbmc import VBMC # VMBC object
# from pyvbmc import VariationalPosterior
# from pyvbmc.priors import SplineTrapezoidal
# from scipy.optimize import minimize
# import scipy.stats as scs

# # Differential equation to be solved
# def diffyqs(X, t, a,b):
#     x, vx = X[0], X[1]
#     dx = vx
#     dv = -x - np.sign(x) * a * np.abs(x)**b
#     return [dx, dv]

# # Initial Condition and time array for solution
# initial_condition = [0.0, 3.5]
# t = np.arange(0,10,0.05)

# # Range of values that parameters can take on
# a_min = 0.0
# a_max = 0.5
# b_min = 1.0
# b_max = 5.0

# # gaussian-ish log_likelihood
# def log_likelihood(theta):
#     a,b = theta
#     sol = odeint(diffyqs, initial_condition, t, args=(a,b))
#     return -np.sum((sol[:,1] - x_true)**2) 


# # Number of parameters (dimension)
# D = 2

# # Bounds for VBMC object (LB and UB expanded a bit beyond what the actual true values could be)
# LB = np.full((1, D), a_min)
# LB[0][0] = a_min - 0.1
# LB[0][1] = b_min - 1
# UB = np.full((1, D), a_max)
# UB[0][0] = a_max + 0.1
# UB[0][1] = b_max + 1
# PLB = np.copy(LB)
# PLB[0][0] = a_min
# PLB[0][1] = b_min
# PUB = np.copy(UB)
# PUB[0][0] = a_max 
# PUB[0][1] = b_max


# # pick a random starting point and do initial minimizatio on it
# np.random.seed(4) # leave this
# x0_rand = np.random.uniform(PLB, PUB)

# #set up prior
# prior = SplineTrapezoidal(LB, PLB, PUB, UB)
# options = {
#     "display": "off"
# }

# print("LB:",LB,"PLB:",PLB,"PUB:",PUB,"UB:",UB,'x0_rand:',x0_rand)

In [5]:
import pandas as pd  # Data handling
from scipy.integrate import odeint # numerical integration
import numpy as np
from pyvbmc import VBMC # VMBC object
from pyvbmc import VariationalPosterior
from pyvbmc.priors import SplineTrapezoidal
from scipy.optimize import minimize
import scipy.stats as scs

# Differential equation to be solved
def diffyqs(X, t, a,b):
    x, vx = X[0], X[1]
    dx = vx
    dv = -x - np.sign(x) * a * np.abs(x)**b
    return [dx, dv]

# Initial Condition and time array for solution
initial_condition = [0.0, 3.5]
t = np.arange(0,10,0.05)

# Range of values that parameters can take on
a_min = 0.0
a_max = 0.5
b_min = 1.0
b_max = 5.0

# gaussian-ish log_likelihood
def log_likelihood(theta):
    a,b = theta
    sol = odeint(diffyqs, initial_condition, t, args=(a,b))
    return -np.sum((sol[:,1] - x_true)**2) 


# Number of parameters (dimension)
D = 2

# Bounds for VBMC object (LB and UB expanded a bit beyond what the actual true values could be)
LB = np.full((1, D), a_min)
LB[0][0] = a_min
LB[0][1] = b_min - 1
UB = np.full((1, D), a_max)
UB[0][0] = a_max + 0.1
UB[0][1] = b_max + 1
PLB = np.copy(LB)
PLB[0][0] = a_min + 0.01
PLB[0][1] = b_min
PUB = np.copy(UB)
PUB[0][0] = a_max 
PUB[0][1] = b_max

# pick a random starting point and do initial minimizatio on it
np.random.seed(4) # leave this
x0_rand = np.random.uniform(PLB, PUB)

#set up prior
prior = SplineTrapezoidal(LB, PLB, PUB, UB)
options = {
    "display": "off"
}

print("LB:",LB,"PLB:",PLB,"PUB:",PUB,"UB:",UB,'x0_rand:',x0_rand)

LB: [[0. 0.]] PLB: [[0.01 1.  ]] PUB: [[0.5 5. ]] UB: [[0.6 6. ]] x0_rand: [[0.48384462 3.188929  ]]


# Running and saving

In [6]:
N = 10
Na = N # adjust number of grid points in a and b separatley if you want
Nb = N

vbmc_count = 0 
param_list = []

for i,a in enumerate(np.linspace(a_min,a_max,Na)):     # a will be y axis in grid
    for j,b in enumerate(np.linspace(b_min,b_max,Nb)): # b will be x axis in grid
        param_list.append([a,b])

        # get truth
        sol_true = odeint(diffyqs, initial_condition, t, args=(a,b))
        x_true = sol_true[:,1]
        
        # Do initial optimization to get an x0
        np.random.seed(3) # change this
        x0 = minimize(
            lambda t: -log_likelihood(t),
            x0_rand,
            bounds=[
                (a_min, a_max),
                (b_min, b_max),
            ],
        ).x

        # run vbmc and save results
        np.random.seed(3) # and this
        vbmc = VBMC(log_likelihood, x0, LB, UB, PLB, PUB, prior = prior, options = options)
        np.random.seed(3) # and this
        vp, results = vbmc.optimize();
        vbmc.save("vbmc" + str(vbmc_count) + ".pkl", overwrite=True)
        vbmc.vp.save("vp" + str(vbmc_count) + ".pkl", overwrite=True)
        vbmc_count += 1

# save parameter values
np.savetxt("param_values.csv", np.array(param_list))

Reshaping x0 to row vector.
vbmc:InitialPointsTooClosePB: The starting points X0 are on or numerically too close to the hard bounds LB and UB. Moving the initial points more inside...
vbmc:InitialPointsOutsidePB. The starting points X0 are not inside the provided plausible bounds PLB and PUB. Expanding the plausible bounds...


/var/folders/w7/05l43x_n3_961q005lwk1dv40000gp/T/ipykernel_85856/922033708.py:18: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  x0 = minimize(
/Users/gracerojo/opt/anaconda3/envs/bpe/lib/python3.11/site-packages/cma/evolution_strategy.py:3829: RuntimeWarning: invalid value encountered in scalar subtract
  current_fitness_range < opts['tolfunrel'] * (es.fit.median0 - es.fit.median_min),
/Users/gracerojo/opt/anaconda3/envs/bpe/lib/python3.11/site-packages/cma/evolution_strategy.py:3829: RuntimeWarning: invalid value encountered in scalar multiply
  current_fitness_range < opts['tolfunrel'] * (es.fit.median0 - es.fit.median_min),


Inference terminated: variational solution stable for options.tol_stable_count fcn evaluations.
Estimated ELBO: -6.026 +/-0.002.
Reshaping x0 to row vector.
vbmc:InitialPointsTooClosePB: The starting points X0 are on or numerically too close to the hard bounds LB and UB. Moving the initial points more inside...
vbmc:InitialPointsOutsidePB. The starting points X0 are not inside the provided plausible bounds PLB and PUB. Expanding the plausible bounds...
Inference terminated: variational solution stable for options.tol_stable_count fcn evaluations.
Estimated ELBO: -6.026 +/-0.002.
Reshaping x0 to row vector.
vbmc:InitialPointsTooClosePB: The starting points X0 are on or numerically too close to the hard bounds LB and UB. Moving the initial points more inside...
vbmc:InitialPointsOutsidePB. The starting points X0 are not inside the provided plausible bounds PLB and PUB. Expanding the plausible bounds...
Inference terminated: variational solution stable for options.tol_stable_count fcn eva

/Users/gracerojo/opt/anaconda3/envs/bpe/lib/python3.11/site-packages/cma/evolution_strategy.py:3823: RuntimeWarning: invalid value encountered in scalar subtract
  current_fitness_range = max(es.fit.fit) - min(es.fit.fit)


Inference terminated: variational solution stable for options.tol_stable_count fcn evaluations.
Estimated ELBO: -4.540 +/-0.002.
Reshaping x0 to row vector.
Inference terminated: variational solution stable for options.tol_stable_count fcn evaluations.
Estimated ELBO: -4.566 +/-0.001.
Reshaping x0 to row vector.
vbmc:InitialPointsOutsidePB. The starting points X0 are not inside the provided plausible bounds PLB and PUB. Expanding the plausible bounds...
Inference terminated: variational solution stable for options.tol_stable_count fcn evaluations.
Estimated ELBO: -4.727 +/-0.000.
Reshaping x0 to row vector.
vbmc:InitialPointsOutsidePB. The starting points X0 are not inside the provided plausible bounds PLB and PUB. Expanding the plausible bounds...
Inference terminated: variational solution stable for options.tol_stable_count fcn evaluations.
Estimated ELBO: -5.030 +/-0.001.
Reshaping x0 to row vector.
vbmc:InitialPointsOutsidePB. The starting points X0 are not inside the provided plau

# Reading everything back in

In [69]:
# reading in saved vps and their associated parameter values
param_values = np.loadtxt("param_values.csv")
vps = []
vbmcs = []
for i in range(len(param_values)-1):
    vps.append(VariationalPosterior.load("vp" + str(i) + ".pkl"))
    vbmcs.append(VariationalPosterior.load("vbmc" + str(i) + ".pkl"))


#unpack into appropriate 2d thing
grid = np.zeros((Na,Nb))
vps_2d = grid.tolist()
params_2d = grid.tolist()
count = 0
for i,a in enumerate(np.linspace(a_min,a_max,1)):     # a will be y axis in grid
    for j,b in enumerate(np.linspace(b_min,b_max,1)): # b will be x axis in grid
        vps_2d[i][j] = vps[count]
        params_2d[i][j] = param_values[count]
        count += 1